# Network Flow optimization
For this exercise, we will use CVXPY which is a Python-embedded modeling language for convex optimization problems. It allows to express problems in a natural way that follows the math, rather than in the restrictive standard form required by solvers (Matlab has the equivalent CVX).

If CVXPY is not already in your environment, you can install it following the instructions reported [here](https://www.cvxpy.org/install/index.html). (for Anaconda in Windows run in your notebook the command **conda install -c conda-forge cvxpy**)

Separable convex network flow optimization arises in several applications, e.g., shortest paths between origin and destination, system optimum network flows, user optimum network flows in homogeneous routing games, electrical current flows in resistor networks.

Given a multigraph $(V,E)$, an **exogenous network flow** is a vector ${\nu} \in \mathrm{R}^V$ such that

\begin{equation}
	\sum_{i \in V} \nu_i = 0.
\end{equation}

A network flow is a vector $f \in \mathrm{R}^E$ satisfying a positivity constraint and a mass conservation constraints, i.e.,

\begin{equation}
 f \ge \mathbf{0}, \quad Bf = \nu.
\end{equation}

Every edge is endowed with a separable non-decreasing convex cost function $\psi_e(f_e)$ such that $\psi_e(0)=0$. 

Given an exogenous flow $\nu$ and a network with node-edge matrix $B$, we study the following optimization problem:

\begin{equation}
\begin{aligned}
	f^* \in \ & \underset{\substack{f \in \mathrm{R}^{E}_+ \\ B f = \nu}}{\arg\min}
	& & \sum_{e \in E} \psi_e (f_e).
\end{aligned}
\end{equation}

The ratio $\psi_e(f_e) / f_e$ may be interpreted as the cost per unit of flow sent along the edge $e$. The convexity of $\psi_e(f_e)$ is thus equivalent to requiring that the marginal cost for sending some flow on each edge is non-decreasing in the flow itself.

# Traffic applications with congestion

## Pigou's example

Pigou was the first to illustrate the role of externality in users' choices, and the difference between user optimum and system optimum flow distributions.

We consider the traffic network with delay functions shown in figure below, which is assumed to
have a unit exogenous inflow at node $o$ and a unit exogenous outflow at node $d$.

![figure](pigou.png)

1.  Determine the social (system) optimum flow distribution using CVXPY. The social optimum flow distribution is defined as the one minimizing the total delay $\sum_{e\in \mathcal{E}} \psi_e(f_e)$, where on every link $e \in \mathcal{E}$, the cost is $\psi_e(f_e) = f_e \tau_e(f_e)$ with $\tau_e(f_e)$ being the delay function. (note that if $\tau_e(f_e)$ is constant, i.e., if there is no congestion, the problem reduces to finding the shortest path)
2. Determine the user optimum flow (Wardrop equilibrium) using CVXPY.
3. Find tolls to align user optimum to social optimum.

### Solution

1.  **Compute social optimum flows.** The social optimum of the network flow problem can be computed with CVXPY using the following code.

In [ ]:
import cvxpy as cp
import numpy as np

# Problem data.
B = np.array([[1, 1],[-1,-1]])
n_edges = B.shape[1]

# exogenous flow vector: one unit of flow enters the origin and 
# exits the destination node
nu = np.array([1, -1])
# Q and l store the coefficients of the quadratic
# and linear terms of the objective function.
# In this case, the objective function is
# f_1^2 + f_2
Q = np.diag([1,0]) # diagonal matrix
l = np.array([0,1]) # 1d array

# Construct the problem
f = cp.Variable(n_edges)
objective = cp.Minimize(cp.quad_form(f,Q) + l.T @ f)
constraints = [B @ f == nu, f >=0]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
cost_opt = prob.solve()
# The optimal value for f is stored in `f.value`.
opt_flow = f.value
print("Social optimal flow:", opt_flow)
print("Optimal cost:", cost_opt)

2.  **Compute Wardrop equilibrium.** 

**Definition**: The Wardrop equilibrium is a path flow distribution such that if a path $p$ is used, then the cost of the path is minimal, i.e.,

$$
z_p>0 \implies c_p(z) \le c_r(z), \quad \forall \ r \in \mathcal{P},
$$

where $\mathcal{P}$ is the path set, $f=Az$ is the projection of the path flow on the link set, and $c_p(z) = \sum_{e: e \in p} \tau_e(f_e)$, namely the cost of a path is the sum of the delays that compose the path.

We say that $f$ is a Wardrop equilibrium if it is induced by a Wardrop equilibrium $z$ via $f=Az$.

**Theorem**: $f^{(0)}$ is a Wardrop equilibrium if and only if it is solution of a network flow optimization given that the cost functions $\psi_e(f_e)$ are chosen as

$$
\psi_e(f_e) = \int_0^{f_e} \tau_e(s)~\mathrm{d}s.
$$
For our problem, the objective function is
$$
\frac{f_1^2}{2}+f_2
$$

In [ ]:
B = np.array([[1, 1],[-1,-1]])
n_edges = B.shape[1]

# exogenous flow vector: one unit of flow enters the origin and 
# exits the destination node
nu = np.array([1, -1])
# Q and l store the coefficients of the quadratic
# and linear terms of the objective function.
Q = np.diag([1/2,0]) # diagonal matrix
l = np.array([0,1]) # 1d array

# Construct the problem.
f = cp.Variable(n_edges)
cp.quad_form(f,Q)
objective = cp.Minimize(cp.quad_form(f,Q) + l.T @ f)
constraints = [B @ f == nu, f >=0]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result_w = prob.solve()
# The optimal value for f is stored in `f.value`.
print("Wardrop equilibrium:", f.value)

Note that the social cost under Wardrop equilibrium is not the solution of the optimization problem above. It is

$$
\sum_{e}f_e^{(0)}\tau_e(f_e^{(0)}),
$$
namely, namely the total travel time at the equilibrium. The Price of Anarchy is

$$
PoA = \frac{\sum_{e}f_e^{(0)}\tau_e(f_e^{(0)})}{\sum_{e}f_e^{*}\tau_e(f_e^{*})}
$$

This observation immediately implies that the price of anarchy cannot be less than 1.

In [ ]:
def cost(f):
    return f[0]*f[0]+f[1] # compute the social cost (total travel time) (f_1)^2+f_2

cost_w = cost(f.value) # evaluate the social cost at the Wardrop equilibrium

print("Wardrop cost:", cost_w)

We can compute the price of anarchy, which is the ratio between the Wardrop cost and the optimal cost.

In [ ]:
PoA = cost_w/cost_opt

print("The price of anarchy:", PoA)

**Theorem** [Roughgarden et al., 2006] The maximum price of anarchy achievable with affine delay functions is $4/3 \sim 1.33333...$.

Thus, this simple example achieves the maximum possible price of anarchy!

![figure](pigou.png)

In other words, what is the price of anarchy? 

- Assume that the user choices are imposed by a central planner. The flow distribution emerging from this centralized decision scheme is the social optimum flow distribution.
- In contrast with this, assume that users are free to choose their path based on other user choices. The flow distribution emerging from this uncoordinated selfish behaviour is the Wardrop equilibrium, in which every user selects a path with minimal delay.

Note indeed that under system optimum flow distribution $(1/2, 1/2)$, the delay associated to each path (in this case, links) are

$$
\tau_1(1/2) = 1/2, \qquad \tau_2(1/2) = 1,
$$

thus an agent that is using the edge 2, if free to choose, will switch to edge 1.

There are many ways to improve the performance of the network. Two examples are:
- to improve the infrastructure (e.g., improving the delay functions of the links, or adding new links), which is called **network design problem**;
- to influence the agents' behaviour, in such a way to align user optimum flows with social optimum flows and reduce the inefficiency due to uncoordinated agent behaviour. This can be done in practice by information design, or adding tolls to the links of the network, which is the subject of the next part.

### How to add tolls: marginal tolls
The property of Wardrop equilibrium is that the cost of each used path (link, in this case) is the same.

However, when switching from edge 2 to edge 1 (same argument may be applied to paths in more complicated networks), an infinitesimal agent produces a modification in the social cost which is

$$
\psi_1'(f_1)-\psi_2'(f_2) = \tau_1(f_1)+f_1 \tau_1'(f_1) - \tau_2(f_2)-f_2 \tau_2'(f_2),
$$

where $f_e \tau_e'(f_e)$ is the derivative of the cost that the other users on edge $e$ will pay due to the addition of infinitesimal flow on edge $e$.

The idea of marginal tolls is that, if a toll equal to $f_e^{*} \tau_e'(f_e^{*})$ is added to each link $e$ (where $f^{*}$ denotes the social optimum flow distribution), then selfish users should pay not only for their delay, but also for the cost that make other users pay due to their choice.

For a general $\psi_e(f_e)$, $$\omega_e^* = \psi_e'(f_e^*) - \tau_e(f_e^*).$$

Thus, users minimize the cost that all the system pays for their choice, which makes the equilibrium coincide with the system optimum.

3. **Compute Wardrop equilibrium with tolls.** Computing the Wardrop equilibrium as in 2., but with the link delays $\tau_e(f_e)$ replaced by $\tau^{(\omega_e)}_e(f_e) = \tau_e(f_e) + \omega_e$, a CVXPY solution can be obtained by the following code:

In [ ]:
B = np.array([[1, 1],[-1,-1]])
n_edges = B.shape[1]

# exogenous flow vector: one unit of flow enters the origin and 
# exits the destination node
nu = np.array([1, -1])

# find marginal tolls: for each edge the toll equals f*_e tau_e'(f_e*)
# for affine delays l_e + b_e f_e, this reads b_e f*_e
# in this example b = [1,0]
omega = opt_flow * np.array([1,0])

# Construct the problem.
f = cp.Variable(n_edges)
objective = cp.Minimize(cp.quad_form(f, Q) + l.T @ f + omega.T @ f)
constraints = [B @ f == nu, f >=0]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for f is stored in `f.value`.
print("Wardrop equilibrium with tolls:", f.value)

The resulting flow vector is $f^{(\omega)} = (1/2, 1/2)$, which is also the social optimum flow, as expected.

## Braess' paradox

Braess' paradox occurs when improving the network, e.g., adding a link, leads to a higher social cost under Wardrop equilibrium, or equivalently, removing a link reduces the social cost under Wardrop equilibrium .

Let us see an example.

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

G = nx.DiGraph()

G.add_nodes_from([0,1,2,3])

G.add_edges_from([(0,1),(0,2),(1,2),(1,3),(2,3)])

pos = {0:[0,0], 1:[2,1], 2:[2,-1], 3:[4,0]}

labels = ['2x','x+1','x','x+1','2x']

zip_operator = zip(G.edges(), labels)
labels = dict(zip_operator)

nx.draw_networkx_nodes(G, pos, node_color ='lightblue')

nx.draw_networkx_labels(G, pos)

nx.draw_networkx_edges(G, pos, arrows=True, width=1)

nx.draw_networkx_edge_labels(G,pos,edge_labels = labels)

plt.show()

Edge labels represent delay functions.

**Social optimum flows**

In [ ]:
B = -nx.linalg.graphmatrix.incidence_matrix(G, oriented=True).toarray()
n_edges = len(G.edges)
# exogenous flow vector: one unit of flow enters the origin and exits the destination node
tau = 1
nu = np.array([1, 0, 0, -1]) * tau

Q = np.diag([2,1,1,1,2]) # diagonal matrix
l = np.array([0,1,0,1,0]) # 1d array

# Construct the problem.
f = cp.Variable(n_edges)
objective = cp.Minimize(cp.quad_form(f,Q) + l.T @ f)
constraints = [B @ f == nu, f >=0]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
cost_opt = prob.solve()
# The optimal value for f is stored in `f.value`.
print("Optimal flows:", f.value)
print("Optimal cost:", cost_opt)

**Wardrop equilibrium**

In [ ]:
B = -nx.linalg.graphmatrix.incidence_matrix(G, oriented=True).toarray()
n_edges = len(G.edges)
# exogenous flow vector: one unit of flow enters the origin and 
# exits the destination node
tau = 1
nu = np.array([1, 0, 0, -1]) * tau
# l stores the coefficients of the linear terms of the objective function, in this case the length of the links.
# In this case, the objective function is

Q = np.diag([1,1/2,1/2,1/2,1]) # diagonal matrix
l = np.array([0,1,0,1,0]) # 1d array

# Construct the problem.
f = cp.Variable(n_edges)
objective = cp.Minimize(cp.quad_form(f,Q) + l.T @ f)
constraints = [B @ f == nu, f >=0]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for f is stored in `f.value`.
print("Wardrop flows:", f.value)

In [ ]:
# cost, defined as \sum_e f_e d_e(f_e)
def cost(f):
    return 2*f[0]*f[0]+f[1]*(f[1]+1)+f[2]*f[2]+f[3]*(f[3]+1)+2*f[4]*f[4]

cost_w = cost(f.value) 

print("Wardrop cost:", cost_w)

PoA = cost_w/cost_opt

print("The price of anarchy:", PoA)

Note that under optimal flows the link $e_3$ is not used, while under Wardrop equilibrium $e_3$ is used, and $PoA>1$.

This suggests that the inefficiency (since PoA>1) may be eliminated by removing the link $e_3$

Let's try!

The optimal flows are not affected by removing $e_3$. Let's compute the Wardrop equilibrium of the new network. 

In [ ]:
# G without $e_3$
G = nx.DiGraph()

G.add_nodes_from([0,1,2,3])

G.add_edges_from([(0,1),(0,2),(1,3),(2,3)])

pos = {0:[0,0], 1:[2,1], 2:[2,-1], 3:[4,0]}

labels = ['2x','x+1','x+1','2x']

zip_operator = zip(G.edges(), labels)
labels = dict(zip_operator)

nx.draw_networkx_nodes(G, pos, node_color ='lightblue')

nx.draw_networkx_labels(G, pos)

nx.draw_networkx_edges(G, pos, arrows=True, width=1)

nx.draw_networkx_edge_labels(G,pos,edge_labels = labels)

plt.show()

In [ ]:
B = -nx.linalg.graphmatrix.incidence_matrix(G, oriented=True).toarray()
n_edges = len(G.edges)

# exogenous flow vector: one unit of flow enters the origin and 
# exits the destination node
tau = 1
nu = np.array([1, 0, 0, -1]) * tau
# l stores the coefficients of the linear terms of the objective function, in this case the length of the links.
# In this case, the objective function is

Q = np.diag([1,1/2,1/2,1]) # diagonal matrix
l = np.array([0,1,1,0]) # 1d array

# Construct the problem.
f = cp.Variable(n_edges)
objective = cp.Minimize(cp.quad_form(f,Q) + l.T @ f)
constraints = [B @ f == nu, f >=0]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for f is stored in `f.value`.
print("Wardrop flows:", f.value)

In [ ]:
# cost, defined as \sum_e f_e d_e(f_e)
def cost(f):
    return 2*f[0]*f[0]+f[1]*(f[1]+1)+f[2]*(f[2]+1)+2*f[3]*f[3]

cost_w = cost(f.value) 

print("Wardrop cost:", cost_w)

PoA = cost_w/cost_opt

print("The price of anarchy:", PoA)

The Wardrop cost decreases, which is the Braess' paradox.

This is an example of network design problem in which links are removed instead of added or improved, exploiting Braess' paradox. Braess' paradox has been observed also in reality, not only in theoretical examples.

**Remark**: the price of anarchy could be minimized also by adding marginal tolls, or by other toll vectors, e.g., by adding a large toll on link $e_3$ in such a way to prevent users to travel along that link.

### A sufficient conditions for PoA=1

**Observation**: assume that delay functions are linear, i.e., $\tau_e(f_e) = a_e f_e$. Then, 

$$
f_e \tau_e(f_e) = a_e f_e^2 = 2 \int_{0}^{f_e} \tau_e(s) ds,
$$

which implies that if the delay functions are linear, the equilibrium and the social optimum solves the same optimization problem, then user equilibria coincide with social optimum flows.

### Resistor networks
Network flow optimization can be adapted to many applications, for instance electrical flows in resistor networks.

In particular, social optimum traffic flows coincide with electrical current in a resistor network if:
- the delay functions are linear, i.e., $\tau_e(f_e) = a_e f_e$;
- the non-negativity constraint $f_e \ge 0$ is relaxed.

The coefficients $a_e$ have to be interpreted as the resistance of the link $e$, and the throughput $\tau$ as the electrical current injected in the network. It can also be proven that the Lagrangian multipliers associated to the mass conservation in the node set corresponds to electrical potential on the nodes of the resistor network. 

# Linear averaging dynamics
In the first part of the lab we study linear averaging dynamics on graphs.

Let $G=(V,E,W)$ be a weighted graph, and $x(t) \in \mathrm{R}^{V}$ denote the state of the nodes of the graph.

The dynamics of $x(t)$ reads

$$
x(t+1) = Px(t),
$$

where $P$ is the normalized adjacency matrix.
Among the applications, the most popular is opinion dynamics, where $x_i$ indicates the opinion of node $i$. This dynamics is known as French - De Groot.

Note that we assume by convention that the opinion of node $i$ is influenced by the opinion of node $j$ if $P_{ij}>0$, i.e., the link $(i,j)$ has to be interpreted as $i$ watching $j$ and updating her opinion based on opinion of $j$.

**Observation**: observe that $\mathbf{1}$ is an equilibrium distribution, since $\mathbf{1} = P \mathbf{1}$ ($P$ is row-stochastic by construction), i.e., consensus distributions are equilibria of the dynamics.

**Questions**: 
- what are the conditions under which the dynamics converges to an equilibrium?
- what are the conditions under which all equilibria are consensus configurations?

**Theorem**: assume that
- its condensation graph has 1 sink;
- the graph is aperiodic.

Then,

$$
\lim_{t \to +\infty} x(t) = \alpha \mathbf{1},
$$

i.e., the agents get to consensus.

## Why aperiodicity matters: example

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

G = nx.Graph()
G.add_edges_from([(0,1),(1,2),(2,3),(3,0)])
n_nodes = len(G)
print("Number of nodes:", n_nodes)

# labels of nodes are couples: (column,row)
pos = {0:[0,0], 1:[0,2], 2:[2,2], 3:[2,0]}
nx.draw(G, pos, with_labels=True)

plt.show()

Note that the graph is periodic because of every cycle has even length (the graph is bipartite and undirected, thus its period is 2)

In [ ]:
# Construct P
W = nx.adjacency_matrix(G) # -> return type is scipy.sparse.csr_matrix
W = W.toarray() # convert A to a numpy array
degrees = np.sum(W,axis=1)
D = np.diag(degrees)
P = np.linalg.inv(D) @ W

In [ ]:
# Assign initial opinion to nodes and run the dynamics
x = np.array([1,0,1,0])

nodecolor=x

# plot centrality at iteration 0
plt.figure(1, figsize=(5,3))
# we draw the graph with same node position "pos" defined above
nx.draw(G,pos,
         nodelist=list(G.nodes()), 
         # node's color reflects centrality values (higher dc = darker color)
         node_color=nodecolor,
         font_size=8,
         # node's colors are on the red scale
         cmap=plt.cm.Reds,
         with_labels=True) 

plt.show()

Nodes with opinion 1 are red, nodes with opinion 0 are white.

After one iteration...

In [ ]:
x = P @ x

nodecolor=x

plt.figure(1, figsize=(5,3))
# we draw the graph with same node position "pos" defined above
nx.draw(G,pos,
         nodelist=list(G.nodes()), 
         # node's color reflects centrality values (higher dc = darker color)
         node_color=nodecolor,
         font_size=8,
         # node's colors are on the red scale
         cmap=plt.cm.Reds,
         with_labels=True) 

print("x(1):", x)

After 5 iterations...

In [ ]:
x = P @ P @ P @ P @ x

nodecolor=x

plt.figure(1, figsize=(5,3))
# we draw the graph with same node position "pos" defined above
nx.draw(G,pos,
         nodelist=list(G.nodes()), 
         # node's color reflects centrality values (higher dc = darker color)
         node_color=nodecolor,
         font_size=8,
         # node's colors are on the red scale
         cmap=plt.cm.Reds,
         with_labels=True) 

print("x(5):", x)

plt.show()

Periodicity of the graph does not allow a proper mixing of the opinions!

Let us add a link to make the graph aperiodic.

In [ ]:
G = nx.Graph()
G.add_edges_from([(0,1),(1,2),(2,3),(3,0)])
G.add_edge(0,2)
n_nodes = len(G)
print("Number of nodes:", n_nodes)

# Construct P
W = nx.adjacency_matrix(G) # -> return type is scipy.sparse.csr_matrix
W = W.toarray() # convert A to a numpy array
degrees = np.sum(W,axis=1)
D = np.diag(degrees)
P = np.linalg.inv(D) @ W

# labels of nodes are couples: (column,row)
nx.draw(G, pos, with_labels=True)

plt.show()

Let us run again the dynamics with same initial conditions as before.

In [ ]:
x = np.array([1,0,1,0])

x = P @ x
print("x(1):", x)

x = P @ x
print("x(2):", x)

x = P @ x
print("x(3):", x)

x = P @ x
print("x(4):", x)

x = P @ x
print("x(5):", x)

x = P @ P @ P @ P @ P @ x
print("x(10):", x)

x = P @ P @ P @ P @ P @ x
print("x(15):", x)

x = P @ P @ P @ P @ P @ x
print("x(20):", x)

x = P @ P @ P @ P @ P @ x
print("x(25):", x)

x = P @ P @ P @ P @ P @ x
print("x(30):", x)

The dynamics reachs an equilibrium, which is a consensus.

### Two questions
- How fast the consensus is achieved?
- What is the consensus value?

We will answer to these questions later on.

## Why 1 sink in the condensation graph: example

In [ ]:
G = nx.DiGraph()
G.add_edges_from([(1,2),(2,1),(1,0),(0,2),(3,2),(3,4),(4,5),(5,4),(6,4),(5,6)])

# labels of nodes are couples: (column,row)
pos = nx.spring_layout(G) 
nx.draw(G, pos, with_labels=True)

plt.show()

How many sink components does this graph have?

Let us compute the condensation graph.

In [ ]:
CG = nx.algorithms.components.condensation(G)

nx.draw(CG)

plt.show()

The condensation graph has 2 sinks. Thus, the sufficient conditions under which the dynamics converges to consensus does not hold.

Let us figure out why by an example.

In [ ]:
# Construct P
W = nx.adjacency_matrix(G) # -> return type is scipy.sparse.csr_matrix
W = W.toarray() # convert A to a numpy array
degrees = np.sum(W,axis=1)
D = np.diag(degrees)
P = np.linalg.inv(D) @ W

# define initial condition
x = [1, 1, 0, 1, 1, 0, 0]

for n in range(100):
    x = P @ x
print("x(100):", x)

The dynamics does not reach consensus, because there are two communities of agents that do not communicate each other, and achieve consensus separately.

In [ ]:
nx.draw(G, pos, with_labels=True)

plt.show()

Let us now add a link in such a way that the condensation graph has now a single sink.

In [ ]:
G.add_edge(6,3)
nx.draw(G, pos, with_labels=True)

plt.show()

In [ ]:
CG = nx.algorithms.components.condensation(G)
nx.draw(CG)

plt.show()

Let us now run the dynamics.

In [ ]:
# Construct P
W = nx.adjacency_matrix(G) # -> return type is scipy.sparse.csr_matrix
W = W.toarray() # convert A to a numpy array
degrees = np.sum(W,axis=1)
D = np.diag(degrees)
P = np.linalg.inv(D) @ W

# define initial condition
x = [1, 1, 0, 1, 1, 0, 0]

for n in range(99):
    x = P @ x
print("x(100):", x, "\n")

for n in range(199):
    x = P @ x
print("x(200):", x, "\n")

for n in range(999):
    x = P @ x
print("x(1000):", x)

We now reach consensus!!

## Consensus value

How is the consensus value computed?

Let $\pi$ denote the normalized left dominant eigenvector of $P$, i.e., the normalized $\pi$ such that $P' \pi = \pi$ (which is unique because the graph has one sink only).
We use the fact that

$$
\pi' x(t) = \pi' P x(t-1) = \pi' x(t-1),
$$

thus $\pi' x(t)$ is constant along the dynamics. Thus,

$$
\pi' x(0) = \pi' \lim_{t \to + \infty} x(t) = \alpha \pi' \mathbf{1} = \alpha.
$$

Thie computation above says that the consensus value $\alpha$ is the weighted average of the initial conditions of the nodes, where the weights are given by the (unique) invariant distribution $\pi$.

Recall that $\pi$ solving $\pi = P' \pi$ is also the invariant distribution centrality, thus the more a node is central the more its initial opinion affects the consensus value.

Let us explore the form of $\pi$ for this example.

In [ ]:
w,v = np.linalg.eig(P.T)

# selects the eigenvalue 1 and print the eigenvector
for index in [i for i in range(len(G)) if np.isclose(w[i],1)]: 
    pi = v[:,index].real  # -> eigenvectors are complex but pi is real, so we convert it to real
    pi = pi/np.sum(pi)
    print("pi", index, "=", pi)
    
nx.draw(G,pos,with_labels=True)

plt.show()

The invariant distribution centrality is 0 for all the nodes that do not belong to the sink of the condensantion graph!

This implies that the initial opinion of the nodes not belonging to the sink are negligible for the consensus value.

The intuition for this is that the nodes 0, 1 and 2 do not care of the other nodes, because are not out-connected to any other node out of the component, and they reach consensus because the induced subgraph on 0,1,2 is aperiodic and strongly connected.
Thus, their evolution is not affected by other nodes. Conversely, the other nodes update their opinion based on 2 also, thus eventually they tend to agree to the opinion of node 2 (and thus 0 and 1 as well).

Let us modify the initial condition of nodes 3,4,5,6 to observe that the consensus value is not affected by this modification.

In [ ]:
# define initial condition
x = [1, 1, 0.5, 50, 50, 8, 12]

for n in range(99):
    x = P @ x
print("x(100):", x, "\n")

for n in range(200):
    x = P @ x
print("x(300):", x, "\n")

for n in range(1000):
    x = P @ x
print("x(1300):", x)

## Speed of convergence

Let us work now for undirected (the following argument holds only for directed) connected graphs. If the graph is also aperiodic, the dynamics is guaranteed to converge to consensus.

Let $\lambda:=\max \{\lambda_2,|\lambda_n|\}$, where $\lambda_1 \ge \lambda_2 \ge \cdots \ge \lambda_n$ are the eigenvalues of $P$. Recall that $\lambda_n \ge -1$ by construction of $P$. 

It is known that the dynamics reaches consensus exponentially fast. In particular, the distance from consensus at time $t$ is in some sense proportional to $\lambda^t$ (as shown in the lecture notes).

Thus, if $\lambda$ is close to $1$, then the convergence is slow, whereas if $\lambda$ is small the convergence is faster.

Note also that for periodic graphs, by Perron-Frobenius theorem have $\lambda_n = -1$ (thus $\lambda=1$), then the convergence to consensus is not achieved. This is consistent with the theory of consensus.

**Remark**: note that every periodic strongly connected graph can be made periodic by adding at least a selfloop in the graph. Indeed, if a selfloop $(i,i)$ is added, then the period of node $i$ is 1. Since all the nodes in the same connected component have same period, then all the nodes have period $1$ and the graph is aperiodic.

To avoid periodic graphs, sometimes it is useful to introduce the **lazy dynamics** obtained by replacing $P$ with 

$$
\frac{P+\mathbf{I}}{2}
$$

This is equivalent to adding selfloops to each node in the graph with weight equivalent to the degree of the node itself.

An interpretation for the lazy dynamics is that nodes have some inertia in the opinion. Instead of averaging over the opinions of the neighbors, they also take into account their opinion at the previous step. In fact,

$$
x_i(t+1) = \frac{x_i(t) + \sum_{j} P_{ij} x_j(t)}{2}.
$$

Let us go back to the initial periodic example, and show that the lazy dynamics converges.

In [ ]:
G = nx.Graph()
G.add_edges_from([(0,1),(1,2),(2,3),(3,0)])

# labels of nodes are couples: (column,row)
pos = nx.spring_layout(G) 
nx.draw(G, pos, with_labels=True)

# Construct P
W = nx.adjacency_matrix(G) # -> return type is scipy.sparse.csr_matrix
W = W.toarray() # convert A to a numpy array
degrees = np.sum(W,axis=1)
D = np.diag(degrees)
P = np.linalg.inv(D) @ W

plt.show()

In [ ]:
Pl = P/2 + np.diag(np.ones(4))/2

x = np.array([1, 0, 1, 0])

for n in range(9):
    x = Pl @ x
print("x(10):", x)

### How fast is the convergence of lazy dynamics?

Note that $\lambda_n(P_{lazy}) = 1/2 + \lambda_n(P)/2 \ge 1/2 + (-1/2) = 0$.

Thus, $\lambda = \lambda_2$. 

In the lazy dynamics, the speed convergence is governed by $\lambda_2$. Let us now define the relaxation time as

$$
\tau_{rel} = \frac{1}{1-\lambda_2}
$$

For undirected graphs, $\lambda_2$ is related to the level of connectedness of the graph. In particular, if the graph is well connected, $\lambda_2$ is smaller and the convergence to consensus is faster.

We do not investigate the details on how to define "connectedness" of graphs in proper way here. However, we can verify by a simple example how connectedness of the graph influences the speed of convergence.

Let us consider a **cycle graph** with 1000 nodes.

For the cycle graph one can show (by the theory of circulant matrices) that

$$
\lambda_2(P) = \cos \left(\frac{2\pi}{n}\right).
$$

Thus,

$$
\lim_{n \to + \infty} \lambda(P_{lazy}) = \frac{1}{2} + \lim_{n \to + \infty} \frac{1}{2} \cos \frac{2\pi}{n} = 1-\frac{\pi^2}{n^2},
$$

and the relaxation time for the cycle is

$$
\tau_{rel} = \frac{n^2}{\pi^2}.
$$

This means that the convergence is achieved exponentially with a rate that scales quadratically with $n$.

In [ ]:
n = 1000
G = nx.cycle_graph(n)

# Construct P
W = nx.adjacency_matrix(G) # -> return type is scipy.sparse.csr_matrix
W = W.toarray() # convert A to a numpy array
degrees = np.sum(W,axis=1)
D = np.diag(degrees)
P = np.linalg.inv(D) @ W

# Construct lazy P
Pl = P/2 + np.diag(np.ones(n))/2

# let us start with random initial conditions
x = np.random.rand(n)

variance = np.var(x)
t=0

while (variance>0.001):
    x = Pl @ x
    t=t+1
    variance = np.var(x)

print('Number of iteration for convergence:', t)

Let us consider the complete graph, which is the most connected graph by definition. For the complete graph,

$$
W=\mathbf{1}\mathbf{1}'-I.
$$

Since $\mathbf{1}\mathbf{1}'$ has equal columns, it has rank $1$. Thus $\mathbf{1}\mathbf{1}'$ has $n-1$ eigenvalues equal to $0$. The remaining eigenvalue can be found by using the fact that the sum of the eigenvalues is the trace of the matrix. Since $\text{trace}(\mathbf{1}\mathbf{1}'-I)=0$, the spectrum of $W$ is

$$
\sigma_W = \{n-1,-1,\cdots,-1\}
$$

Since the complete graph is regular and every node has degree $n-1$, $P=W/(n-1)$
Thus, the spectrum of $P$ is

$$
\sigma_P = \{1,-\frac{1}{n-1},\cdots,-\frac{1}{n-1}\},
$$

and $P_{lazy}$ has spectrum

$$
\sigma_{P_{lazy}} = \{1,-\frac{1}{2(n-1)}+\frac{1}{2},\cdots,-\frac{1}{2(n-1)}+\frac{1}{2}\}
$$

This implies that $\lambda_2 = \frac{1}{2}-\frac{1}{2(n-1)} = \frac{n-2}{2(n-1)}$ and the relaxation time for large $n$ in the complete graph tends to

$$
\lim_{n \to +\infty} \tau_{rel} = 2,
$$

i.e., even for infinite $n$ the relaxation time is finite.

In [ ]:
G = nx.complete_graph(n)

# Construct P
W = nx.adjacency_matrix(G) # -> return type is scipy.sparse.csr_matrix
W = W.toarray() # convert A to a numpy array
degrees = np.sum(W,axis=1)
D = np.diag(degrees)
P = np.linalg.inv(D) @ W

# Construct lazy P
Pl = P/2 + np.diag(np.ones(n))/2

# let us start with random initial conditions
x = np.random.rand(n)

variance = np.var(x)
n=0

while (variance>0.001):
    x = Pl @ x
    n=n+1
    variance = np.var(x)

print('Number of iteration for convergence:', n)

### How fast is the convergence of (not lazy) averaging dynamics?

In lazy dynamics the speed of convergence is described by $\lambda_2$. In standard averaging dynamics, also $\lambda_n$ plays a role. We recall indeed that the speed of convergence is described by

$$
\lambda := \max\{\lambda_2,|\lambda_n|\},
$$

where $\lambda_n$ is the smallest eigenvalue of $P$.

While $\lambda_2$ is related to the level of connectedness of the graph, we shall see in a qualititive manner the role of $\lambda_n$ by examples.

In [ ]:
# let us construct a regular graph with 6m nodes, and degree of each node equal to m
m = 5
G = nx.cycle_graph(6*m)

n_nodes = 6*m

for n in range(6*m):
    for i in range(m):
        G.add_edge(n,(6*(i+1)+n-3) % (6*m))

pos = nx.circular_layout(G)

nx.draw(G,pos,with_labels=True)

# Construct P
W = nx.adjacency_matrix(G) # -> return type is scipy.sparse.csr_matrix
W = W.toarray() # convert A to a numpy array
degrees = np.sum(W,axis=1)
D = np.diag(degrees)
P = np.linalg.inv(D) @ W

plt.show()

In [ ]:
w,v = np.linalg.eig(P)
w = w.real

print(w)

The graph is bipartite, hence $-1$ is in the spectrum and the convergence to consensus is not guaranteed.

Let us now add a single edge to break the periodicity while mantaining the total number of edges.

In [ ]:
G.add_edge(0,2)
G.remove_edge(0,9)

nx.draw(G,pos,with_labels=True)

# Construct P
W = nx.adjacency_matrix(G) # -> return type is scipy.sparse.csr_matrix
W = W.toarray() # convert A to a numpy array
degrees = np.sum(W,axis=1)
D = np.diag(degrees)
P = np.linalg.inv(D) @ W

w,v = np.linalg.eig(P)
w = w.real

print(w)

The graph is not periodic now. However, $|\lambda_n| \sim 1$, which implies that the convergence of the dynamics is slow. On the other hand, $\lambda_2$ is "small" ($\sim 1/2$), thus the lazy dynamics converges quickly.

In some sense, $|\lambda_n| \sim 1$ reflects the fact that the graph is almost periodic. Let us see this with an example, by comparing the speed in convergence of the lazy dynamics and the standard dynamics.

In [ ]:
# construct a larger graph
m = 50
G = nx.cycle_graph(6*m)

G.add_edge(0,2)

n_nodes = 6*m

for n in range(6*m):
    for i in range(m):
        G.add_edge(n,(6*(i+1)+n-3) % n_nodes)
        

# Construct P
W = nx.adjacency_matrix(G) # -> return type is scipy.sparse.csr_matrix
W = W.toarray() # convert A to a numpy array
degrees = np.sum(W,axis=1)
D = np.diag(degrees)
P = np.linalg.inv(D) @ W

# let us start with random initial condition and standard dynamics
x0 = np.random.rand(G.number_of_nodes())>1/2
x = x0

variance = np.var(x)
t=0

while (variance>0.00001):
    x = P @ x
    t=t+1
    variance = np.var(x)
    if t>100000:
        t='inf'
        break
        
print('Number of iterations for convergence of standard dynamics:', t)

# same initial condition, lazy dynamics
x = x0

# Construct lazy P
Pl = P/2 + np.diag(np.ones(G.number_of_nodes()))/2

variance = np.var(x)
t=0

while (variance>0.00001):
    x = Pl @ x
    t=t+1
    variance = np.var(x)
    if t>100000:
        t='inf'
        break
        
print('Number of iterations for convergence of lazy dynamics:', t)

## Wisdom of crowds
Consider a graph, and assume that state of each node represents a noisy estimate of the real state $\mu$, i.e.,

$$
x_i = \mu + y_i,
$$

with $E[y_i]=0$, and the variance $\sigma^2 (y_i) = \sigma^2$ for each $i$.

Assume that the graph is connected and aperiodic Eventually, the agents will reach consensus, i.e., $\lim_{t \to +\infty} x(t) = \alpha \mathbf{1}$. 

**Question**: what is the consensus value $\alpha$?

Notice that $\alpha = \pi' (\mu \mathbf{1} + y) = \mu + \pi'y$, then

$$
E[\alpha] = \mu + \pi' E[y] = \mu
$$

so the final estimate of the network is unbiased. Moreover,

$$
\quad \sigma_{\alpha}^2 = \sigma^2 \sum_{i} \pi_i^2 < \sigma^2,
$$

because $\sum_{i} \pi_i^2 <1$ unless the graph has a unique sink node. The interesting observation is that the estimate $\alpha$ has a smaller variance than $\sigma$, i.e., the crowd is able to reconstruct a more precise estimate of the real state than the single agents of the graph.

Let us verify this on a complete graph, where $\pi_i = 1/n$, thus $\sigma_\alpha^2 = \sigma^2/n$. 

In [ ]:
G = nx.complete_graph(100)

# Construct P
W = nx.adjacency_matrix(G) # -> return type is scipy.sparse.csr_matrix
W = W.toarray() # convert A to a numpy array
degrees = np.sum(W,axis=1)
D = np.diag(degrees)
P = np.linalg.inv(D) @ W

# start with random initial states and run the dynamics 200 times
# store in alfa_err the consensus values at each run
alfa_err = np.zeros(200)

for i in range(200):
# rand returns uniformly random values in [0,1], thus mu = 1/2
    x = np.random.rand(100)
    for n in range(500):
        x = P @ x
    alfa_err[i] = (1/2 - np.mean(x))*(1/2 - np.mean(x))

print("Variance of the node states:", 1/12)
print("Variance of the consensus state:", np.mean(alfa_err), "\n")

As expected the variance of $\alpha$ is about $1/100$ of the original variance.

Note that $\sum_{i} \pi_i^2$ tends to 1 if one node has almost all the centrality, and is minimal when the nodes have the same centrality (as in the complete graph, or in the cycle graph). This implies that if a graph is more democratic, then the consensus algorithm leads to better estimates of the true state. If a few nodes have all the centralities, the consensus value is less reliable.

Let us see this with a another example.

In [ ]:
G = nx.cycle_graph(10)
G = nx.Graph.to_directed(G)
G.remove_edges_from([(0,1),(0,9)])
G.add_edge(0,0)

nx.draw(G, with_labels=True)

# Construct P
W = nx.adjacency_matrix(G) # -> return type is scipy.sparse.csr_matrix
W = W.toarray() # convert A to a numpy array
degrees = np.sum(W,axis=1)
D = np.diag(degrees)
P = np.linalg.inv(D) @ W

plt.show()

In [ ]:
# start with random initial states and run the dynamics
alfa_err = np.zeros(200)

for i in range(200):
# rand returns random values in [0,1], thus \mu = 1/2
    x = np.random.rand(10)
    var = np.var(x)
    for n in range(500):
        x = P @ x
    alfa_err[i] = (1/2 - np.mean(x))*(1/2 - np.mean(x))

print("Expected variance of the node states:", 1/12)
print("Empirical variance of the consensus state:", np.mean(alfa_err), "\n")

In this graph the consensus value is exactly the initial state of node $0$, because the condensation graph has 1 sink only, which is node $0$.

$$
\pi = \delta^{(0)}.
$$

This graph is the opposite of the complete graph, in the sense that the invariant distribution centrality is totally on node $0$. Thus the variance of the consensus state equals the variance of the single node.

Note that this argument holds independently of the size of the graph.

### Application: distributed computation of average

Let the node set describe a set of sensors that are deployed in some regions in order to collect measurements of some quantity of interest (for example, the temperature). 

Assume that these sensors have limited communication and computation capabilities that allow each of them to exchange information only with those other sensors that are close enough in space. 

Let the graph $G = (V, E)$ describe the pattern of closeness among the sensors $i$ and $j$ so that there is an undirected link between node $i$ and node $j$ if they can communicate to each other (possibly using link weights decreasing with distance). Then, one can design a distributed algorithm for computing the average of the sensor's measurements based on the averaging dynamics.

Let $x_i(0)$ be the measurement of each node. 

We are interested in designing an iterative distributed algorithm that allows the nodes to compute

$$
x = \frac{1}{n}\sum_i x_i(0)
$$

**First attempt**: we run a consensus algorithm. Since the graph is undirected, $\pi_i = \frac{w_i}{\sum_j w_j}$. Thus, the algorithm converges to a consensus $\alpha \mathbf{1}$ such that

$$
\alpha = \sum_i \pi_i x_i(0) = \sum_{i} \frac{w_i}{w} x_i(0).
$$

If each edge knows its degree $w_i$, each node can rescale its initial state, i.e., $y_i(0) = \frac{x_i(0)}{w_i}$. The consensus algorithm for the variable $y_i$ thus converges to

$$
\alpha_y = \sum_{i} \frac{w_i}{w} y_i(0) = \frac{1}{w} \sum_{i} x_i(0).
$$

If we assume that each node knows the average degree of the network, thus

$$
x = \alpha_y \frac{w}{n} = \alpha_y \overline{w}.
$$


In [ ]:
G = nx.karate_club_graph()

# Fix node positions on all pictures according to spring layout
pos = nx.spring_layout(G) 
nx.draw_networkx(G, pos)

n_nodes = len(G)

x = np.random.rand(n_nodes)

plt.show()

In [ ]:
# Let us run the consensus algorithm for y

# Construct P
W = nx.adjacency_matrix(G) # -> return type is scipy.sparse.csr_matrix
W = W.toarray() # convert A to a numpy array
degrees = np.sum(W,axis=1)
D = np.diag(degrees)
P = np.linalg.inv(D) @ W

y = x/degrees

for t in range(1000):
    y = P @ y

print("Average state:", np.mean(x))
# choose arbitrarly the first node, but all the nodes reach consensus on y
print("Average computed distributively", y[0] * np.sum(degrees) / n_nodes)

The algorithm works! Unfortunately, requiring that each node knows the average degree of the network is not realistic (and not distributed).

However, there exists another way to solve the problem.

**Second attempt**: we run a second averaging dynamics, with initial condition $z_i(0) = \frac{1}{w_i}$.

This converges to

$$
\alpha_z = \lim_{t \to + \infty} z_i(t) = \sum_i z_i(0) \frac{w_i}{w} = \sum_{i} \frac{1}{w} = \frac{1}{\overline{w}}
$$

By combining the two, each node can estimate the average estimate by 

$$
\frac{\lim_{t \to + \infty} y_i(t)}{\lim_{t \to + \infty} z_i(t)} = \frac{\alpha_y}{\alpha_z} = \frac{x}{\overline{w}} \cdot \overline{w} = x.
$$

This method does not require any global knowledge on the network and it is distributed.

In [ ]:
# Let us implement this

z = 1/degrees

for t in range(1000):
    z = P @ z

print("average state:", np.mean(x))
# choose arbitrarly the first node, but all the nodes reach consensus both on y and z
print("average computed distributively", y[0] / z[0])

# Linear averaging dynamics with stubborn agents
We study how to simulate the linear averaging dynamics on graphs in presence of stubborn agents.

We focus on the optimal placement problem, which consists of optimally chosing the position of a stubborn node on the graph in order to maximize its influence on the asymptotic outcome of the dynamics.

Let us first summarize the theory in presence of stubborn agents.

We are given a network, where the agents $V$ are divided in regular agents $R$ and stubborn agents $S$. The regular agents update their opinion $x(t)$ according to the standard DeGroot model, while the stubborn agents do not update their opinions, so that $u(t) \equiv u$.

Let $Q=P|_{R \times R}$ and $E=P|_{R \times S}$.

Thus, the dynamics for the regular agents read

$$
x(t+1) = Qx(t) + Eu.
$$

Under some assumptions (see the lecture notes for details) the dynamics converges to 

$$
x^* = (\mathbf{I}-Q)^{-1}Eu.
$$

Note that, in contrast with the dynamics without stubborn nodes:
- the asymptotic state is not a consensus;
- the asymptotic state does not depend on the initial opinions.

### Implementation

We start by implementing the averaging dynamics with stubborn nodes. 

To illustrate this procedure, we will analyse the following example that involves a $3 \times 4$ grid graph $\mathcal G$.

In [ ]:
G = nx.generators.lattice.grid_graph(dim=[3,4])
n_nodes = len(G)
print("Number of nodes:", n_nodes)

# labels of nodes are couples: (column,row)
nx.draw_spectral(G, with_labels=True)

print(G.nodes())

plt.show()

In [ ]:
# Construct a dictionary that maps the label of nodes  
# (from (0,0) to (2,1)) to their index (from 0 to n_nodes-1)
indices = dict()
for i in range(n_nodes):
    indices[list(G.nodes)[i]] = i
print(indices)

In [ ]:
# Number of iterations
n_iter = 50;
    
# Stubborn and regular nodes
stubborn = [(0,0), (3,2)];
stubborn_id = [indices.get(key) for key in stubborn]
regular = [node for node in G.nodes if node not in stubborn]
regular_id = [id for id in range(n_nodes) if id not in stubborn_id]
print("Stubborn nodes:", stubborn, "\n")
print("Regular nodes:", regular, "\n")

In [ ]:
# Input to stubborn nodes
u = [0,1]

# P matrix
A = nx.adjacency_matrix(G) # -> return type is scipy.sparse.csr_matrix
A = A.toarray() # convert A to a numpy array
degrees = np.sum(A,axis=1)
D = np.diag(degrees)
P = np.linalg.inv(D) @ A

# Submatrices
# Using ix_ one can construct index arrays that will 
# index a cross product. 
# a[np.ix_([1,3],[2,5])] returns the array [[a[1,2] a[1,5]], [a[3,2] a[3,5]]].
Q = P[np.ix_(regular_id, regular_id)]
E = P[np.ix_(regular_id, stubborn_id)]

# Sample a random initial condition for regular nodes
ic = np.random.uniform(0,1,len(regular))

# Set the initial condition for the dynamics
x = np.zeros((n_nodes,n_iter))
x[stubborn_id,0] = u;
x[regular_id,0] = ic;
print("Initial state:", x[:,0], "\n")

In [ ]:
# Evolve the opinion vector
for t in range(1,n_iter):
    x[regular_id, t] = Q @ x[regular_id, t-1] + E @ x[stubborn_id, t-1]
    x[stubborn_id, t] = x[stubborn_id, t-1];

print("Final state:")
x_final = x[:,n_iter-1]
for key in indices.keys():
    print(key, x_final[indices[key]])

In [ ]:
fig = plt.figure(1, figsize=(7,7))
ax = plt.subplot(111)

for node in range(n_nodes):
    trajectory = x[node,:]
    ax.plot(trajectory, label='node {0:d}'.format(node))
    
ax.legend()

plt.show()

In [ ]:
average = np.average(x_final)
print("Average asymptotic opinion:", average)

As expected, the dynamics does not converge to consensus. Moreover, in contrast with averaging without input stubborn nodes, we can verify that the asymptotic equilibrium does not depend on the initial condition. Furthermore, the dynamics converge to an equilibrium even though the graph is periodic.

In [ ]:
# Sample another random initial condition for regular nodes
ic = np.random.uniform(0,1,len(regular))

x = np.zeros((n_nodes,n_iter))
x[stubborn_id,0] = u;
x[regular_id,0] = ic;
print("Initial state:", x[:,0], "\n")

# Evolve the opinion vector
for t in range(1,n_iter):
    x[regular_id, t] = Q @ x[regular_id, t-1] + E @ x[stubborn_id, t-1]
    x[stubborn_id, t] = x[stubborn_id, t-1];

print("Final state:")
x_final = x[:,n_iter-1]
for key in indices.keys():
    print(key, x_final[indices[key]])
    
fig = plt.figure(1, figsize=(7,7))
ax = plt.subplot(111)

for node in range(n_nodes):
    trajectory = x[node,:]
    ax.plot(trajectory, label='node {0:d}'.format(node))
    
ax.legend()

plt.show()

## Optimal placement of stubborn nodes
Suppose that node $(0,0)$ is stubborn with opinion $u_{(0,0)}=0$. We want to find the optimal position $(i,j)$ of a stubborn node with opinion $1$ in order to maximize the asymptotic average opinion.

A very simple approach is to consider all possible positions $(i,j)$ and pick the best one.

In [ ]:
# Number of iterations
n_iter = 50;

# We will store final opinion vectors and 
# average of final opinions in dictionaries
# where the key is the position (i,j) of the 
# 1-stubborn agent
final_opinions = dict()
average_opinion = dict() 


for (i,j) in G.nodes:
    # Position (0,0) is occupied by the 0-stubborn node
    if (i,j)==(0,0):
        continue
        
    # Stubborn and regular nodes
    stubborn = [(0,0), (i,j)];
    stubborn_id = [indices.get(key) for key in stubborn]
    regular = [node for node in G.nodes if node not in stubborn]
    regular_id = [id for id in range(n_nodes) if id not in stubborn_id]
    print("Stubborn nodes:", stubborn)

    # Input to stubborn nodes
    u = [0,1]


    # P matrix
    A = nx.adjacency_matrix(G) # -> return type is scipy.sparse.csr_matrix
    A = A.toarray() # convert A to a numpy array
    degrees = np.sum(A,axis=1)
    D = np.diag(degrees)
    P = np.linalg.inv(D) @ A

    # Submatrices
    Q = P[np.ix_(regular_id, regular_id)]
    E = P[np.ix_(regular_id, stubborn_id)]

    # Sample a random initial condition for regular nodes
    ic = np.random.uniform(0,1,len(regular))

    # Set the initial condition for the dynamics
    x = np.zeros((n_nodes,n_iter))
    x[stubborn_id,0] = u;
    x[regular_id,0] = ic;

    for t in range(1,n_iter):
        x[regular_id, t] = Q @ x[regular_id, t-1] + E @ x[stubborn_id, t-1]
        x[stubborn_id, t] = x[stubborn_id, t-1];

    final_opinions[(i,j)] = x[:,n_iter-1]
    average_opinion[(i,j)] = np.average(final_opinions[(i,j)])
    print("Average opinion:", average_opinion[(i,j)], "\n")

To visualize the dependence of the average asymptotic opinion on the position of the $1$-stubborn node we can plot the grid graph by setting each node's size and color according to the magnitude of the average asymptotic opinion when the $1$-stubborn is placed in such node.

In [ ]:
# add a dummy (0,0) entry to the dictionary
# to make its size = n_nodes
average_opinion[(0,0)] = 0

plt.figure(1, figsize=(7,3))
nx.draw(G, 
        pos = nx.spectral_layout(G),
        with_labels=True, 
        node_size = [np.exp(10*average_opinion[node]) for node in G.nodes],
        node_color= [average_opinion[node] for node in G.nodes],
        font_size=8,
        # node's colors are on the red scale
        cmap=plt.cm.Reds)

plt.show()

The optimal placements of the 1-stubborn player are the maximizers of the final average opinion:

In [ ]:
# convert the average opinion values from dict_values to numpy array
avg = np.fromiter(average_opinion.values(),dtype=float)

optimal_place = [place for place in average_opinion.keys() if average_opinion[place]==np.max(avg)]
print("Optimal placements:", optimal_place, "\n")

# print the final opinions under optimal placement
opt_final = final_opinions.get(*optimal_place)
print("Final opinions after optimal placement:", opt_final)

In [ ]:
# plot the asymptotic opinions of the nodes when the stubborn is placed in (1,1)

plt.figure(1, figsize=(7,3))
nx.draw(G, 
        pos = nx.spectral_layout(G),
        with_labels=True, 
        node_size = [np.exp(8*opt_final[indices.get(node)]) for node in G.nodes],
        node_color= [opt_final[indices.get(node)] for node in G.nodes],
        font_size=8,
        # node's colors are on the red scale
        cmap=plt.cm.Reds)

plt.show()

In [ ]:
### Back to an old example

In [ ]:
G = nx.cycle_graph(10)
G = nx.Graph.to_directed(G)
G.remove_edges_from([(0,1),(0,9)])
G.add_edge(0,0)

nx.draw(G, with_labels=True)

n_nodes = G.number_of_nodes()

# Construct P
W = nx.adjacency_matrix(G) # -> return type is scipy.sparse.csr_matrix
W = W.toarray() # convert A to a numpy array
degrees = np.sum(W,axis=1)
D = np.diag(degrees)
P = np.linalg.inv(D) @ W

n_iter = 100
x = np.zeros((10,n_iter))

# set initial condition (1,0,0,0,0,0,0,0,0,0)
x[0,0] = 1

# evolve the states
for t in range(1,n_iter):
    x[:,t] = P @ x[:,t-1]

print("Average final opinions:", np.mean(x[:,n_iter-1]))

plt.show()

In [ ]:
fig = plt.figure(1, figsize=(7,7))
ax = plt.subplot(111)

for node in range(G.number_of_nodes()):
    trajectory = x[node,:]
    ax.plot(trajectory, label='node {0:d}'.format(node))
    
ax.legend()

plt.show()

We can prove that this is equivalent to having node 0 stubborn with opinion 1.

Indeed, note that because of the topology of the graph, the opinion of node 0 is not influenced by anyone. The same dynamics can be obtained by assuming that node 0 is stubborn.

In [ ]:
# Stubborn and regular nodes
stubborn = [0];
regular = [node for node in G.nodes if node not in stubborn]

print("Stubborn nodes:", stubborn, "\n")
print("Regular nodes:", regular, "\n")

# Input to stubborn nodes
u = [1]

# Submatrices
Q = P[np.ix_(regular, regular)]
E = P[np.ix_(regular, stubborn)]

# Set the initial condition for the dynamics
x = np.zeros((n_nodes,n_iter))
x[stubborn,0] = u;
print("Initial condition:", x[:,0], "\n")

# Evolve the opinion vector
for t in range(1,n_iter):
    x[regular, t] = Q @ x[regular, t-1] + E @ x[stubborn, t-1]
    x[stubborn, t] = x[stubborn, t-1];

x_final = x[:,n_iter-1]
print(x_final)

In [ ]:
fig = plt.figure(1, figsize=(7,7))
ax = plt.subplot(111)

for node in range(G.number_of_nodes()):
    trajectory = x[node,:]
    ax.plot(trajectory, label='node {0:d}'.format(node))
    
ax.legend()

plt.show()

Same trajectory as before!

### A more general model: overview on Friedkin-Johnsen model
The French-DeGroot dynamics can be generalized by taking into account that each agents is not completely regular or completely stubborn. The opinion of each agents is in part due to "innate" opinions, and in part due to influence of society.

The following opinion dynamics model is known as Friedkin-Johnsen model.
Here:
- $x_i(t)$ is the opinion of the agent $i$;
- $y_i$ is the innate opinion of agent $i$.
- $\alpha_i$ is its level of stubborness, i.e., the level of confidence in his/her opinion $y_i$.

The dynamics reads:

$$
x_i(t+1) = \alpha_i y_i + (1-\alpha_i) \sum_{j} P_{ij} x_j(t).
$$

If $\alpha = \mathbf{0}$, we get the French-DeGroot model without input.

If $\alpha \in \{0,1\}^{V}$, we get the French-DeGroot model with stubborn nodes.

As the French-DeGroot model with input, also the Friedkin-Johnsen dynamics converges to a non-consensus state, which depends on $\alpha, y$, but not on the initial opinions $x(0)$.